In [ ]:
import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import pandas as pd


In [ ]:
start = time.time()


# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At Bellis' home
#topdir = 'bnv_plambda'

# On Bellis' laptop
topdir = './'

filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


In [ ]:
nlam = data['nLambda0']
lamfl = data['Lambda0FlightLen']


d1idx = data['Lambda0d1Idx']
d2idx = data['Lambda0d2Idx']

d1lund = data['Lambda0d1Lund']
d2lund = data['Lambda0d2Lund']

Bd1idx = data['Bd1Idx']
Bd2idx = data['Bd2Idx']

Bd1lund = data['Bd1Lund']
Bd2lund = data['Bd2Lund']



plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(nlam);

plt.subplot(1,2,2)
plt.hist(nlam[nlam!=2]);

In [ ]:
print(lamfl)
print()

print(d1idx)
print(d1lund)
print()

print(d2idx)
print(d2lund)
print()

print(Bd1idx)
print(Bd1lund)
print()

print(Bd2idx)
print(Bd2lund)
print()

In [ ]:
mask = lamfl >= 0

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(ak.flatten(lamfl),bins=100);

plt.subplot(1,2,2)
plt.hist(ak.flatten(lamfl[mask]), bins=100);

In [ ]:
mask = lamfl >= 0

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.hist(ak.num(lamfl));

plt.subplot(1,2,2)
plt.hist(ak.num(lamfl[mask]));

In [ ]:
mask = nlam == 2

x1 = d1idx[mask][:,0]
x2 = d1idx[mask][:,1]

x3 = d2idx[mask][:,0]
x4 = d2idx[mask][:,1]

x5 = Bd2idx[mask][:,0]
x6 = Bd2idx[mask][:,1]


print(x1)
print(x2)
print()

print(len(x1))
print(len(x1[x1 != x2]))

print(len(x3))
print(len(x3[x3 != x4]))

print(len(x5))
print(len(x5[x5 != x6]))


In [ ]:
mask = nlam == 2

print(d1idx[mask])

x1 = d1idx[mask][:,1]

x1